In [65]:
load("../framework/instance_gen.sage")
load("../framework/geometry_twist.sage")
load("../framework/geometry.sage")
demo = False
from scipy.io import loadmat
import random

In [66]:
def data_to_measure(sca_seeds, params, param, frodo_distribution): 
    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]
    return measures
def simu_measured(secret, measures):
    """
    This fonction simulates the information gained by
    Bos et al attack. For a given secret, the simulation
    is outputting a score table at random from the data of Bos et al.
    :secret: an integer being the secret value
    :measurement: a score table
    """
    if secret in measures.keys():
        measurement = random.choice(measures[secret])
    else: 
        measurement = None
    return measurement

def measurement_to_aposteriori(measurement):
    """
    This fonction transforms a score table into an aposteriori distribution.
    According to the matlab code of Bos et al. attack,
    the score table is proportional
    to the logarithm of the aposteriori probability. We thus apply the exponential
    and renormalize.
    :measurement: a score table
    :apost_dist: a dictionnary of the aposteriori distribution
    """
    if measurement is not None:
        s = sum([exp(meas) for meas in measurement])
        measurement = [exp(meas)/s for meas in measurement]
        apost_dist = {}
        for key_guess in range(-len(frodo_distribution) + 1, len(frodo_distribution)):
            apost_dist[key_guess] = measurement[key_guess + len(frodo_distribution) - 1]
    else:
        apost_dist = None
    return apost_dist


In [70]:
def var_to_prob(var, mean, D_s, x):
    discrete_prob_list = [gaussian_prob(var, mean, x) for x in D_s.keys()]
#     print(discrete_prob_list)
    norm = sum(discrete_prob_list)

    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
    return ex/denom/norm
def gaussian_prob(var, mean, x):
    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
#     print(var, mean, x, denom, ex)
    return 1/denom * ex

In [71]:
load("../framework/instance_gen.sage")
demo = False
from scipy.io import loadmat
import random

"""  Example
Uncomment the following to get an example
of the detailed computation (without redundancy)
"""
upper_bound_as_prob = True
guess_at_end = True
# demo = True
# logging("--- Demonstration mode (no redundancy of the attacks) ---")

for params in [ 'CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:
    logging("Set of parameters: " + params)

    if params == 'NIST1':
        # NIST1 FRODOKEM-640
        n = 640
        m = 640
        q = 2**15
        frodo_distribution = [9456, 8857, 7280, 5249, 3321,
                              1844, 898, 384, 144, 47, 13, 3]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        # We used the following seeds for generating Bos et al. data
        # These seeds were generated with the matlab code genValues.m
        sca_seeds = [42, 72, 163, 175, 301, 320, 335, 406, 430, 445]
        param = 4

    elif params == 'NIST2':
        # NIST2 FRODOKEM-976
        n = 976
        m = 976
        q = 65536
        frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
                              396, 118, 29, 6, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [74, 324, 337, 425, 543, 1595, 1707, 2026, 2075, 2707]
        param = 5

    elif params == 'CCS1':
         
        frodo_distribution = [22528, 15616, 5120, 768]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        param = 0

    elif params == 'CCS2':
        n = 592
        m = 592
        q = 2 ** 12
        frodo_distribution = [25120, 15840, 3968, 384, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 3, 4, 5, 6, 7, 8, 9, 14, 16]
        param = 1


    elif params == 'CCS3':
        n = 752
        m = 752
        q = 2 ** 15
        frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 4, 5, 7, 9, 10, 12, 13, 14, 15]
        param = 2

    elif params == 'CCS4':
        n = 864
        m = 864
        q = 2 ** 15
        frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [256, 393, 509, 630, 637, 652, 665, 1202, 1264, 1387]
        param = 3


    """  Original Security   """

    A, b, ebdd = initialize_kannan_ellipsoid_from_LWE(EBDD_non_homo,
                                                          n,q, m, D_s, D_s,
                                                          homogeneous=False, verbosity=0)
    
#     ebdd.integrate_q_vectors(q, indices=range(n, n + m))
#     (beta, _) = ebdd.estimate_attack()
#     logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")

    """  Refined Side channel attack  """

    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]


####################Initial Phase#######################
    if demo:
        A, b, ebdd = initialize_kannan_ellipsoid_from_LWE(EBDD_non_homo,
                                                          n,q, m, D_s, D_s,
                                                          homogeneous=False, verbosity=0)
    
          
        measured = [simu_measured(ebdd.u[0, i], measures) for i in range(n)]
        ###TO FIX###
        if upper_bound_as_prob:
            estimate_SCA_prob(50, ebdd, measured, 0.9, D_s, D_s)
        else:
            estimate_SCA_num(50, ebdd, measured, 100, D_s, D_s)
        ###TO FIX###
    else:
        """  Averaging
        The following averages the measures to get accurate data
        for the paper. The averaging mode is quite long.
        """
        nb_tests_per_params = 2

        # Chosen values for the number of guesses
        if upper_bound_as_prob:
            max_guesses = 0.9
        else:
            if params == 'CCS1':
                max_guesses = 175 + 100
            elif params == 'CCS2':
                max_guesses = 300 + 150
            elif params == 'CCS3':
                max_guesses = 250 + 125
            elif params == 'CCS4':
                max_guesses = 250 + 125
            elif params == 'NIST1':
                max_guesses = 100 + 75
            elif params == 'NIST2':
                max_guesses = 250 + 125

        
        proba_success = 0
        guesses_average = 0
        
        beta_origin_mean = 0
        beta_post_mean = 0
        beta_guess_delete_mean = 0
        beta_intersect_mean = 0

        dvol_origin_mean = 0
        dvol_post_mean = 0
        dvol_guess_delete_mean = 0
        dvol_intersect_mean = 0
        
        beta_track = {}
        dvol_track = {}
        
        for i in range(nb_tests_per_params):
            A, b, ebdd = initialize_kannan_ellipsoid_from_LWE(EBDD_non_homo,
                                                                   n,
                                                                   q, m, D_s, D_s, homogeneous=False, verbosity=0)
    
            measured = [simu_measured(ebdd.u[0, i], measures) for i in range(n)]
            if upper_bound_as_prob:
                beta_list, dvol_list, p_success, guesses = estimate_SCA_prob_guess_end(None, ebdd, n, m, 
                                                  measured, max_guesses, D_s, D_s, to_print = True)
            else:
                beta_list, dvol_list, p_success, guesses = estimate_SCA_num_guess_end(None, ebdd, n, m,
                                                  measured, max_guesses, D_s, D_s, to_print = True)

            beta_origin_mean += beta_list[0]
            beta_post_mean += beta_list[1]
            beta_guess_delete_mean += beta_list[3]
            beta_intersect_mean += beta_list[2]
            
            dvol_origin_mean += dvol_list[0]
            dvol_post_mean += dvol_list[1]
            dvol_guess_delete_mean += dvol_list[3]
            dvol_intersect_mean += dvol_list[2]
            
            proba_success += p_success
            guesses_average += guesses
            
            beta_track[i] = beta_list
            dvol_track[i] = dvol_list
    
        beta_origin_mean /= nb_tests_per_params
        beta_post_mean /= nb_tests_per_params
        beta_guess_delete_mean /= nb_tests_per_params
        beta_intersect_mean /= nb_tests_per_params

        dvol_origin_mean /= nb_tests_per_params
        dvol_post_mean /= nb_tests_per_params
        dvol_guess_delete_mean /= nb_tests_per_params
        dvol_intersect_mean /= nb_tests_per_params
        
        proba_success /= nb_tests_per_params
        guesses_average /= nb_tests_per_params
        
        logging("Attack: %3.2f bikz" % beta_origin_mean, style="HEADER")
        logging("Attack after SCA hints: %3.2f bikz" % beta_post_mean, style="HEADER")
        logging("Attack after SCA,LWE intersection: %3.2f bikz" % beta_intersect_mean, style="HEADER")
        logging("Attack after guesses: %3.2f bikz" % beta_guess_delete_mean, style="HEADER")
        
        logging("Attack: %3.2f dvol" % dvol_origin_mean, style="HEADER")
        logging("Attack after SCA hints: %3.2f dvol" % dvol_post_mean, style="HEADER")
        logging("Attack after SCA,LWE intersection: %3.2f dvol" % vol_intersect_mean, style="HEADER")
        logging("Attack after guesses: %3.2f dvol" % dvol_guess_delete_mean, style="HEADER")
        
        logging("Number of guesses: %4d" % guesses_average, style="HEADER")
        logging("Success probability: %3.2f" %proba_success, style="HEADER")


 Set of parameters: CCS1 
 Attack without hints:  269.07 bikz 
scaled@funct posteriori_integration? suppose False:  False
 Attack with post distributions:  156.58 bikz 
scaled@funct posteriori_integration? suppose False:  False
ebdd.S should be close to Sigma_SCA
s_s, ebdd.S (rescaling to <= 1 21/16 64.3039401859067
ebdd.S should be close to Sigma_SCA
s_s, ebdd.S (rescaling to <= 1 21/16 72.17849726590222
ebdd.S should be close to Sigma_SCA
s_s, ebdd.S (rescaling to <= 1 21/16 76.97419744379143
ebdd.S should be close to Sigma_SCA
s_s, ebdd.S (rescaling to <= 1 21/16 39.05681466574925
ebdd.S should be close to Sigma_SCA
s_s, ebdd.S (rescaling to <= 1 21/16 80.27790836560675
ebdd.S should be close to Sigma_SCA
s_s, ebdd.S (rescaling to <= 1 21/16 86.64684231873564
ebdd.S should be close to Sigma_SCA
s_s, ebdd.S (rescaling to <= 1 21/16 87.68482679088059
ebdd.S should be close to Sigma_SCA
s_s, ebdd.S (rescaling to <= 1 21/16 68.42950713096755
ebdd.S should be close to Sigma_SCA
s_s, ebdd

rank 352 352 352 352
<class 'ValueError'>
f(a) and f(b) must have different signs
Using min/max as fallback...


/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/scipy/optimize/_minimize.py:761: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


poly(pi_sol) = -81.29929826442776
poly(1, 0) = -256399.03502351412 -81.29929825490836
2.4519887255103345e-05
6.965877061108905e-08
(Solution) pi = 6.17725262760070e-11
alpha = 0.999999999955662
VOL OF ELLIP_OUTPUT:  -238.880566255039
check if mu's from above are computed the same.
kannan_ellipsoid_mean_update version [               -4525223242107463/4503599627370496               12492732292517/18446744073709551616                 4784460288808783/4503599627370496  395125250051426354715299/37778931862957161709568 -194124548371260363640577/37778931862957161709568]
direct version [               -4525223242107463/4503599627370496               12492732292517/18446744073709551616                 4784460288808783/4503599627370496  395125250051426354715299/37778931862957161709568 -194124548371260363640577/37778931862957161709568]
ellip_norm: 
38.14100927399466
comparison computations on Sigma, should be very close
ebdd.S
[                    2196484954012693/1623313478486440542208         

AssertionError: 

In [67]:
def estimate_SCA_prob_guess_end(report_every, ebdd, n, m, measured, max_guesses, D_s, D_e, to_print):
    #Upper bounding the maximum number of guessed coordinate
    """ 
    This function evaluates the security loss after Bos et al attack
    :report_every: an integer that give the frequency of
    logging (None for no logging)
    :ebdd: instance of the class EBDD
    :measured: table representing the (simulated) information
    given by Bos et al attack
    :max_guesses: integer for upperbounding the number of guesses
    """

    MINIMUM_VAR = 0.5
    proba_best = {}
    Id = identity_matrix(n + m)
    list_av = [0] * n
    list_var = [0] * n
    
    beta_list = []
    dvol_list = []
    
    #Attack without hints
    (beta_origin, _) = ebdd.estimate_attack()
    dvol_origin =  ebdd.dvol
    beta_list += [beta_origin]
    dvol_list += [dvol_origin]
    
    intersect_indices = []
    
    logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")
    _, s_s = average_variance(D_s)
    for i in range(n):
        apost_dist = measurement_to_aposteriori(measured[i])
        if apost_dist is not None:
            proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
            av, var = average_variance(apost_dist)
            av = float(av)
            var = float(var)
            list_av[i] = av
            list_var[i] = var
            if av*av > s_s - var:
#                 gap_list += [av*av - (var_prev - var)] #gap is how much posterior ellipsoid exceeds prior ellipsoid
                intersect_indices += [i]
        else:
            av = None
            var = None
        v = vec(Id[i])
        if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
            verbose = 2 
        else:
            verbose = 0
        ebdd.verbosity = verbose
        if verbose == 2:
            logging("[...%d]" % report_every, newline=False)
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
        if var == 0:
            logging("     Real Perfect Hint     ", style="HEADER")
            list_var[i] = MINIMUM_VAR

    #####Posteriori Phase####
    integrate_posteriori_dist(ebdd, A, b, q, list_av, list_var, D_s)
    (beta_post, _) = ebdd.estimate_attack()
    logging("Attack with post distributions:  %3.2f bikz" % beta_post, style="HEADER")
    dvol_post =  ebdd.dvol
    beta_list += [beta_post]
    dvol_list += [dvol_post]
    #####Intersection Phase#####
    _, _ = integrate_ellipsoid_intersection(ebdd, A, b, q, list_av, list_var, D_s, D_e, intersect_indices, to_print)
    (beta_inter, _) = ebdd.estimate_attack()
    dvol_inter =  ebdd.dvol
    logging("Attack with intersection integration:  %3.2f bikz" % beta_inter, style="HEADER")
    beta_list += [beta_inter]
    dvol_list += [dvol_inter]
    #####Guess Phase####
    
    if report_every is not None:
        logging("     Hybrid attack estimation     ", style="HEADER")
    
    if to_print:
        print("first 5x5 submatrix of ebdd.S after intersection")
        print(ebdd.S[:5, :5])
        ebdd.S[i, i]
    intersect_var = list(list(ebdd.S[i, i] for i in range(n)))
    intersect_av = list(ebdd.mu[0][:n])
    proba_best2 = {}
    
    for k in range(len(intersect_var)):
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
            continue
        if var == 0:
            v = vec(Id[k])
            logging("     Real Perfect Hint after Intersection Phase    ", style="HEADER")
            intersect_var[k] = MINIMUM_VAR
            ebdd.S[k] = MINIMUM_VAR
            ebdd.integrate_perfect_hint(v, av, estimate=verbose)
        #choose intersect_av[k] to be the selected value is assuming the distribution is gaussian, so the mean has the best prob.
        proba_best2[k] = var_to_prob(intersect_var[k], intersect_av[k], D_s, intersect_av[k]) 
        
    sorted_guesses = sorted(proba_best2.items(),
                            key=lambda kv: - kv[1])

    sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))]

   
    proba_success = 1.
    ebdd.verbosity = 0
    guesses = 0
    j = 0
    guess_indices = []
    guess_vals = []
    for t in sorted_guesses:
        j += 1
        if proba_success * proba_best2[t] >= max_guesses: #(guesses <= max_guesses):(guesses <= max_guesses):
            guess_indices += [t]
            guess_vals += [proba_best2[t]]
            guesses += 1
            proba_success *= proba_best2[t]
    if to_print:
        print("guess_indices: ", guess_indices)
        print("guess_vals: ", guess_vals)
    ebdd.integrate_perfect_entries_hints(guess_indices, guess_vals, to_print = to_print) 
    (beta_guess, _) = ebdd.estimate_attack()
    dvol_guess =  ebdd.dvol
    beta_list += [beta_guess]
    dvol_list += [dvol_guess]
    logging("Attack after guesses:  %3.2f bikz" % bete_guess, style="HEADER")
#     beta_before_last_q = ebdd.beta
#     if report_every is not None:
#         ebdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
#     else:
#         ebdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
    return beta_list, dvol_list, proba_success, guesses


In [68]:
def integrate_posteriori_dist(ebdd, A, b, q, list_av, list_var, D_e):
#   needs_ellipsoid_scaling(scale=False)
    print("scaled@funct posteriori_integration? suppose False: ", ebdd.scaled)
    
    m = A.nrows()
    n = A.ncols()
    mu_e, s_e = average_variance(D_e)
    
    mu_s_post = vec(list_av)
    
    #Update mu
    ebdd.mu = kannan_ellipsoid_mean_update(A, b, mean_s = mu_s_post, mean_e = vec([mu_e]*m))
    
    #Update S
    #1. Compute Sigma_s_e
    zero1 = zero_matrix(ZZ, m, n)
    Sigma_s_e = diagonal_matrix(list_var + [s_e]*m)
    #2. Compute S
    upleft_mat_inv = block4(identity_matrix(n), A.T/q, zero1, 1/q*identity_matrix(m)) #upleft_mat.inverse()
    ebdd.S = upleft_mat_inv.T * Sigma_s_e * upleft_mat_inv    

    return

In [69]:
def integrate_ellipsoid_intersection(ebdd, A, b, q, list_av, list_var, D_s, D_e, intersect_indices, to_print):

    """
    Increase LWE ellipsoid in the bad coordinates to ensure the intersections
    Sigma_SCA, Sigma_SCA--posteriori
    mu_LWE, mu_LWE--prior
    
    """
    #needs_ellipsoid_scaling(scale=False)
    print("scaled@funct posteriori_integration? suppose False: ", ebdd.scaled)
    
    
    m = A.nrows()
    n = A.ncols()
    mu_s, s_s = average_variance(D_s)
    mu_e, s_e = average_variance(D_e)
    
    #ellipsoids of first n dimension, rescaling to <= 1
    Sigma_SCA = diagonal_matrix(list_var)*n
    mu_SCA = vec(list_av)
    Sigma_LWE = matrix(Sigma_SCA)
    mu_LWE = matrix(mu_SCA)
   
    for i in intersect_indices:
        if to_print:
            print("ebdd.S should be close to Sigma_SCA")
            print("s_s, ebdd.S (rescaling to <= 1", s_s, ebdd.S[i,i]*n)
            assert(ebdd.S[i,i]*n - Sigma_SCA[i,i] < 10**-5)
        Sigma_LWE[i, i] = s_s*n
        mu_LWE[0, i] = mu_s
        
    
#     c_vec = c_SCA * factor
#     sub_denom = scal(c_vec * W_SCA * c_vec.T)
#     W_SCA_extend = W_SCA - 1/(1 + sub_denom) * W_SCA * c_vec.T * c_vec * W_SCA
    if to_print:
        print("VOL OF ELLIP_LWE: ", ln(Sigma_LWE.det()).n())
        print("VOL OF ELLIP_SCA: ", ln(Sigma_SCA.det()).n())
    
    mu_s, Sigma_s = ellipsoid_intersection(mu_LWE, Sigma_LWE, mu_SCA, Sigma_SCA, tolerance=1e-10)
    print("VOL OF ELLIP_OUTPUT: ", ln(Sigma_s.det()).n())
    
    ####update mean#####
    zero1 = zero_matrix(ZZ, m, n)
    upleft_mat = block4(identity_matrix(n), -A.T, zero1, q*identity_matrix(m))
    b_new = b - mu_s * A.T
    b0 = concatenate(zero_matrix(QQ, 1, n), -b_new)
    upleft_mat_inv = upleft_mat.inverse()
    mu_s_temp = b0 * upleft_mat_inv
    mu_temp = concatenate(mu_s, mu_s_temp[0][n:])
    ####update mean#####
    ebdd.mu = kannan_ellipsoid_mean_update(A, b, mean_s = mu_s, mean_e = vec([mu_e]*m))
    if to_print:
        print("check if mu's from above are computed the same.")
        print("kannan_ellipsoid_mean_update version", ebdd.mu[0, n-3: n+2])
        print("direct version", mu_temp[0, n-3: n+2])
        
    #left to be done
    Sigma_s_inv = matrix(np_inv(Sigma_s * 1/n)) # convert to <= dim
    B_new_sub = matrix(scipy.linalg.cholesky(Sigma_s_inv)) #lower=True

    B_hybrid_eps = block4(B_new_sub, zero_matrix(ZZ, n, m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, m, m))
    E_hybrid_eps = B_hybrid_eps * B_hybrid_eps.T * (1/n) #<= 1
    B_Sigma = matrix(scipy.linalg.cholesky(matrix(np_inv(ebdd.S))))
    B_Sigma_eps = block4(zero_matrix(ZZ, n, n), B_Sigma[0:n, n:], B_Sigma[n:, 0:n], B_Sigma[n:, n:]) #<=dim
    
    new_S_inv = (n)/(m + n) * E_hybrid_eps + (1 - (n)/(m + n)) * (1/(m))*B_Sigma_eps*B_Sigma_eps.T    #<= 1          
    S_temp = matrix(np_inv(new_S_inv * (n + m))) #<= dim
    if to_print:
        print("ellip_norm: ")
        print(scal((ebdd.u - ebdd.mu) * matrix(np_inv(ebdd.S))/(n + m) * ((ebdd.u - ebdd.mu)).T))
    
    #Update S
    #1. Compute Sigma_s_e
    zero1 = zero_matrix(ZZ, m, n)
    Sigma_e = diagonal_matrix([s_e]*m)
    Sigma_s_e = block4(Sigma_s*1/n, zero1.T, zero1, Sigma_e) 
    
    #2. Compute S
    upleft_mat_inv = block4(identity_matrix(n), A.T/q, zero1, 1/q*identity_matrix(m)) #upleft_mat.inverse()
    ebdd.S = upleft_mat_inv.T * Sigma_s_e * upleft_mat_inv #<=dim
    if to_print:
        print("comparison computations on Sigma, should be very close")
        print("ebdd.S")
        print(ebdd.S[n-2:n+3, n-2:n+3], ebdd.S.rank())
        print("S_temp")
        print(S_temp[n-2:n+3, n-2:n+3], S_temp.rank())
        print("scaled for output@funct posteriori_integration? suppose False: ", ebdd.scaled)
    
    
    return mu_s, Sigma_s  


In [26]:
diagonal_matrix([1]+[2])

[1 0]
[0 2]

In [24]:
[aaa[i][0] for i in range(len(aaa))]


[1, 2]

In [ ]:
To fix: 
    (done) 1. initialize_kannan_ellipsoid_from_LWE_sca
        a. Kannan_embedding
    2. estimate_SCA_prob_guess_first 
    (done)a. post integration
    (done)b. intersect integration
        c. add ellipsoid norm printout after 1 is fixed.
        d. check scaled after 1 is fixed
    3. q_vector
    (done)4. dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n)
    (done)5. Initiate EBDD use B.?
Current:
    2.
    c. add ellipsoid norm printout after 1 is fixed.
    d. check scaled after 1 is fixed
Finish:
    1. perfect entries hint
    2. Initialize_kannan_ellipsoid_from LWE
        a. Kannan_embedding done
    3. estimate_SCA_prob_guess_first 
        a. post integration
        b. intersect integration (4. reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n))
        d. check scaled after 1 is fixed

In [18]:
def var_to_prob(var, mean, D_s, x):
    discrete_prob_list = [gaussian_prob(var, mean, x) for x in D_s.keys()]
#     print(discrete_prob_list)
    norm = sum(discrete_prob_list)

    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
    return ex/denom/norm
def gaussian_prob(var, mean, x):
    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
#     print(var, mean, x, denom, ex)
    return 1/denom * ex

In [1]:
K = random_matrix(ZZ,10); K

[ -1   1   0   1  -4   0   0   0  -1   1]
[  1  -2  -1   0 -28  -3   0   0  -2  -4]
[ -2   0   8  13   1   3   0  -1  -1   1]
[  8   0   1  -3  -1  -2  -3  99   3   1]
[ -6  -1   2   1  -1  -1  10  -1  -6   0]
[ -1   6  -1  -2   0   0  -4   1 -23   1]
[  0   2   1  17  -1   1   1  -2   2   1]
[ -1  -2  -2  -1   0  -1   2  -2   0  14]
[ -2   0   1  -5  -1   0  -1  -3   1  -1]
[  3   2   0   2   0   1   0  -2  -1  -9]

In [2]:
K1 = K.inverse()

In [14]:
K[:,3]

[ 1]
[ 0]
[13]
[-3]
[ 1]
[-2]
[17]
[-1]
[-5]
[ 2]

In [15]:
alist = [1,2]

In [17]:
type(K[:,3]*alist[0])

<class 'sage.matrix.matrix_integer_dense.Matrix_integer_dense'>

In [18]:
vec([1,2])

[1 2]